# 음성데이터 텍스트 변환

In [ ]:
# %%
# 1. 필수 라이브러리 설치
# 이 셀은 한 번만 실행하면 됩니다.
# !pip install pandas
# !pip install git+https://github.com/m-bain/whisperX.git
# !pip install pyannote.audio==3.1.1

In [2]:
# %%
# 2. 라이브러리 임포트 및 기본 설정
import os
import torch
import whisperx
from whisperx.diarize import DiarizationPipeline
import pandas as pd
from dotenv import load_dotenv

# .env 파일 로드
# 노트북 파일과 같은 디렉토리에 .env가 있거나, 상위 디렉토리에 있는 .env를 찾습니다.
load_dotenv()

# Hugging Face 인증 토큰 불러오기
HF_TOKEN = os.getenv("HF_TOKEN")

# 토큰 존재 여부 확인
if HF_TOKEN is None:
    raise ValueError("Hugging Face 토큰이 설정되지 않았습니다. .env 파일을 확인하세요.")
else:
    print("Hugging Face 토큰을 성공적으로 불러왔습니다.")

# 처리할 오디오 파일 경로
audio_file = r"D:\workspace\woogawooga_project\datas\금감원_보이스피싱 체험관_그놈 목소리_수사기관 사칭형\4FF_.mp3"

# CSV 파일을 저장할 디렉토리 경로
output_dir = r"D:\workspace\woogawooga_project\dataset"

# 저장 디렉토리가 없으면 생성
os.makedirs(output_dir, exist_ok=True)

print("라이브러리 임포트 및 기본 설정 완료.")
print(f"입력 파일: {audio_file}")
print(f"출력 디렉토리: {output_dir}")


# %%
# 3. WhisperX 및 화자 분리 모델 로드
device = "cuda" if torch.cuda.is_available() else "cpu"
compute_type = "float16" if torch.cuda.is_available() else "int8"
batch_size = 16

print(f"사용 장치: {device}")

# Whisper 모델 로드
print("Whisper 모델 로드 중...")
model = whisperx.load_model("large-v2", device, compute_type=compute_type)
print("Whisper 모델 로드 완료.")

# 화자 분리 모델 로드
print("화자 분리 모델 로드 중...")
diarize_model = DiarizationPipeline(use_auth_token=HF_TOKEN, device=device)
print("화자 분리 모델 로드 완료.")


# %%
# 4. 음성 처리 실행
print("오디오 파일 로드 중...")
audio = whisperx.load_audio(audio_file)

print("음성 텍스트 변환 중...")
result = model.transcribe(audio, batch_size=batch_size)

print("타임스탬프 정렬 중...")
model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)
result = whisperx.align(result["segments"], model_a, metadata, audio, device, return_char_alignments=False)

print("화자 분리 및 할당 중...")
diarize_segments = diarize_model(audio)
result = whisperx.assign_word_speakers(diarize_segments, result)

print("음성 처리 완료.")


# %%
# 5. 결과 처리 및 저장
if "segments" in result and result["segments"]:
    final_transcript = []
    current_turn = None

    for segment in result["segments"]:
        speaker = segment.get('speaker', 'UNKNOWN')
        
        if current_turn is None:
            current_turn = {'speaker': speaker, 'text': segment['text'], 'start': segment['start'], 'end': segment['end']}
        elif current_turn['speaker'] == speaker:
            current_turn['text'] += ' ' + segment['text']
            current_turn['end'] = segment['end']
        else:
            final_transcript.append(current_turn)
            current_turn = {'speaker': speaker, 'text': segment['text'], 'start': segment['start'], 'end': segment['end']}
    
    if current_turn is not None:
        final_transcript.append(current_turn)

    df = pd.DataFrame(final_transcript)

    base_filename = os.path.splitext(os.path.basename(audio_file))[0]
    csv_path = os.path.join(output_dir, f"{base_filename}.csv")
    
    df.to_csv(csv_path, index=False, encoding='utf-8-sig')

    print(f"성공적으로 CSV 파일 저장 완료: {csv_path}")
    display(df)
else:
    print("처리할 대화 내용이 없습니다.")

Hugging Face 토큰을 성공적으로 불러왔습니다.
라이브러리 임포트 및 기본 설정 완료.
입력 파일: D:\workspace\woogawooga_project\datas\금감원_보이스피싱 체험관_그놈 목소리_수사기관 사칭형\4FF_.mp3
출력 디렉토리: D:\workspace\woogawooga_project\dataset
사용 장치: cpu
Whisper 모델 로드 중...
No language specified, language will be first be detected for each audio file (increases inference time).
>>Performing voice activity detection using Pyannote...


Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.5.2. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint d:\workspace\woogawooga_project\.venv\lib\site-packages\whisperx\assets\pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.7.1+cpu. Bad things might happen unless you revert torch to 1.x.
Whisper 모델 로드 완료.
화자 분리 모델 로드 중...
화자 분리 모델 로드 완료.
오디오 파일 로드 중...
음성 텍스트 변환 중...
Detected language: ko (0.97) in first 30s of audio...
타임스탬프 정렬 중...


d:\workspace\woogawooga_project\.venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kwj90\.cache\huggingface\hub\models--kresnik--wav2vec2-large-xlsr-korean. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to

화자 분리 및 할당 중...


d:\workspace\woogawooga_project\.venv\lib\site-packages\pyannote\audio\models\blocks\pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\ReduceOps.cpp:1839.)
  std = sequences.std(dim=-1, correction=1)


음성 처리 완료.
성공적으로 CSV 파일 저장 완료: D:\workspace\woogawooga_project\dataset\4FF_.csv


,speaker,text,start,end
0,SPEAKER_02,여보세요?,0.487,25.162
1,SPEAKER_04,안녕하세요.,25.182,25.302
2,SPEAKER_02,서울중앙지검 첨단범죄수사팀 장민호 수사관입니다. 뭐라고요? 장자 민자 우자요. 누구...,25.322,28.006
3,UNKNOWN,네.,28.026,28.066
4,SPEAKER_04,무슨 일인지 모르는 거야. 연루된 사건 때문에 연락드렸는데요. 어떤 거요?,28.086,28.907
5,SPEAKER_02,본인과 일로 된 사건에요. 무슨 조건이요? 맹리도형 사건입니다. 맹리도형인데? 네...,30.068,58.553
6,SPEAKER_01,얼마 전 저희 검찰에서 김철수 조건으로 된 금융본부의 사기집단을 검거했습니다.,58.573,59.434
7,SPEAKER_02,금관 현장에서 본인 명으로 된 농촌 통장하고 국민의 통장이 발견돼서 지금 수사하고...,60.123,85.232
8,UNKNOWN,네.,85.252,85.292
9,SPEAKER_02,그래서 본인 앞으로 연락드리는 이유가 이 통장을 본인이 직접 개설하셔서 김철수 씨한...,85.312,87.775


In [ ]:
# %%
# 📌 [1] 라이브러리 임포트 및 초기 설정
import os
import torch
import whisperx
from whisperx.diarize import DiarizationPipeline
import pandas as pd
from dotenv import load_dotenv

# .env 환경변수 파일 로드 (Hugging Face 토큰 등 사용)
load_dotenv()

# Hugging Face 토큰 불러오기
HF_TOKEN = os.getenv("HF_TOKEN")
if HF_TOKEN is None:
    raise ValueError("Hugging Face 토큰이 설정되지 않았습니다.")
else:
    print("Hugging Face 토큰을 성공적으로 불러왔습니다.")

# 분석 대상 오디오 파일 경로
audio_file = r"D:\workspace\woogawooga_project\datas\금감원_보이스피싱 체험관_그놈 목소리_수사기관 사칭형\4FF_.mp3"

# 결과 CSV 저장 경로
output_dir = r"D:\workspace\woogawooga_project\dataset"
os.makedirs(output_dir, exist_ok=True)  # 디렉토리 없으면 생성

print("라이브러리 임포트 및 기본 설정 완료.")
print(f"입력 파일: {audio_file}")
print(f"출력 디렉토리: {output_dir}")

In [ ]:
# %%
# 📌 [2] WhisperX 및 화자 분리 모델 로드
device = "cuda" if torch.cuda.is_available() else "cpu"
compute_type = "float16" if device == "cuda" else "int8"
batch_size = 16

print(f"사용 장치: {device}")

# WhisperX 음성 인식 모델 로드
print("Whisper 모델 로드 중...")
model = whisperx.load_model("large-v2", device, compute_type=compute_type)
print("Whisper 모델 로드 완료.")

# 화자 분리 모델 로드 (Hugging Face 인증 필요)
print("화자 분리 모델 로드 중...")
diarize_model = DiarizationPipeline(use_auth_token=HF_TOKEN, device=device)
print("화자 분리 모델 로드 완료.")

In [ ]:
# %%
# 📌 [3] 음성 파일 처리 및 화자 분리
print("오디오 파일 로드 중...")
audio = whisperx.load_audio(audio_file)

# 음성을 텍스트로 변환 (batch 단위)
print("음성 텍스트 변환 중...")
result = model.transcribe(audio, batch_size=batch_size)

# 타임스탬프 정렬 (단어 정렬 정확도 향상)
print("타임스탬프 정렬 중...")
model_a, metadata = whisperx.load_align_model(
    language_code=result["language"], device=device
)
result = whisperx.align(
    result["segments"], model_a, metadata, audio, device, return_char_alignments=False
)

# 화자 분리 및 화자 정보 할당
print("화자 분리 및 할당 중...")
diarize_segments = diarize_model(audio)
result = whisperx.assign_word_speakers(diarize_segments, result)

print("음성 처리 완료.")

In [ ]:
# %%
# 📌 [4] 결과 처리 및 CSV 저장
if "segments" in result and result["segments"]:
    final_transcript = []
    current_turn = None  # 현재 화자의 발화 저장용

    for segment in result["segments"]:
        speaker = segment.get("speaker", "UNKNOWN")  # speaker 키가 없을 경우 대비

        # 동일 화자일 경우 한 문장으로 이어붙이기
        if current_turn is None:
            current_turn = {
                "speaker": speaker,
                "text": segment["text"],
                "start": segment["start"],
                "end": segment["end"],
            }
        elif current_turn["speaker"] == speaker:
            current_turn["text"] += " " + segment["text"]
            current_turn["end"] = segment["end"]
        else:
            # 화자가 바뀌면 이전 발화 저장 후 새로 시작
            final_transcript.append(current_turn)
            current_turn = {
                "speaker": speaker,
                "text": segment["text"],
                "start": segment["start"],
                "end": segment["end"],
            }

    # 마지막 발화도 저장
    if current_turn is not None:
        final_transcript.append(current_turn)

    # DataFrame으로 변환
    df = pd.DataFrame(final_transcript)

    # 출력 파일명 구성
    base_filename = os.path.splitext(os.path.basename(audio_file))[0]
    csv_path = os.path.join(output_dir, f"{base_filename}.csv")

    # CSV 저장 (utf-8-sig로 저장하여 엑셀에서도 한글 깨짐 방지)
    df.to_csv(csv_path, index=False, encoding="utf-8-sig")

    print(f"성공적으로 CSV 파일 저장 완료: {csv_path}")
    display(df)

else:
    print("처리할 대화 내용이 없습니다.")